In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2,3"
from debate import DebateNode
from paper_details import Paper
from persona import PaperAuthor
from moderator import Moderator
import argparse
from typing import List
from vllm import LLM
import os
import json
from data_pairer import parse_papers
import numpy as np

/home/ishikaa2/miniconda3/envs/tod/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-07 14:58:39,086	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
class Args():
    def __init__(self):
        self.log_dir = "logs"
        self.focus_paper = "https://arxiv.org/pdf/1706.03762"
        self.cited_paper = "https://arxiv.org/pdf/1810.04805"
        self.topic = "language model architectures"

args = Args()

In [3]:
if not os.path.exists(args.log_dir):
    os.makedirs(args.log_dir)

# if not os.path.exists("data.json"):
parse_papers(args.focus_paper, args.cited_paper)
with open('data.json', 'r') as file:
    data = json.load(file)

In [4]:
model_server = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct",tensor_parallel_size=2, gpu_memory_utilization=0.5, max_num_seqs=100) #,enable_prefix_caching=True)

INFO 11-07 14:58:51 config.py:905] Defaulting to use mp for distributed inference
WARNING 11-07 14:58:51 arg_utils.py:957] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 11-07 14:58:51 config.py:1021] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 11-07 14:58:51 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.57it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.62it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.93it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.67it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.78it/s]



INFO 11-07 14:58:59 model_runner.py:1067] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=2755133) INFO 11-07 14:59:00 model_runner.py:1067] Loading model weights took 7.5122 GB
INFO 11-07 14:59:00 distributed_gpu_executor.py:57] # GPU blocks: 13892, # CPU blocks: 4096
INFO 11-07 14:59:00 distributed_gpu_executor.py:61] Maximum concurrency for 131072 tokens per request: 1.70x
(VllmWorkerProcess pid=2755133) INFO 11-07 14:59:02 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
(VllmWorkerProcess pid=2755133) INFO 11-07 14:59:02 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-07 14:59:02 model_

In [5]:
item = data[0]
f_pap = item['focus']
c_pap = item['cited']

focus_paper = PaperAuthor(
    model = model_server,
    paper = Paper(f_pap),
    focus=True,
    id=0
)

cited_paper = PaperAuthor(
    model = model_server,
    paper = Paper(c_pap),
    focus=False,
    id=1
)

In [7]:
focus_paper.paper.retrieve_top_k("transformers", k=5)

ValueError: shapes (1024,) and (768,) not aligned: 1024 (dim 0) != 768 (dim 0)

In [7]:
moderator = Moderator(model_server)

paper_authors = [focus_paper, cited_paper]
leaf_node_label = args.topic

if args.log_dir != "":
    with open(os.path.join(args.log_dir, 'self_deliberation.txt'), 'w') as f:
        f.write(f'Topic: {args.topic}\n\n')

# each node has a topic
root_node = DebateNode(leaf_node_label)
# subtrees = root_node.conduct_self_deliberation(leaf_node_label, paper_authors, log=args.log_dir) # k new, finer topics to discuss

In [8]:
topic = args.topic
log = None
focus_paper = None
arguments = []
children = []
evidence = {}

for paper_author in paper_authors:
    # gather evidence
    evidence, scores = paper_author.gather_evidence(topic, return_scores=True)

    if paper_author.id not in evidence.keys(): evidence[paper_author.id] = []
    evidence[paper_author.id].append(evidence)

    # develop k arguments
    if paper_author.id not in arguments.keys(): arguments[paper_author.id] = []
    author_args = paper_author.generate_arguments(topic, evidence,k=2)
    arguments[paper_author.id].append(author_args)

    # check if paper is the focus
    if paper_author.focus:
        focus_paper = paper_author

    # logging
    if log is not None:
        with open(os.path.join(log, 'self_deliberation.txt'), 'a') as f:
            f.write(f'Topic: {topic}\n\n')
            f.write(f'Gather Evidence:\n\n')
            temp = "\n".join([f'{s} - {e}' for s, e in zip(scores, evidence)])
            f.write(f'{paper_author.focus} paper:\n{temp}\n\n')

            f.write(f'Develop Arguments:\n\n')
            f.write(f'{paper_author.focus} paper:\n{author_args}\n\n')

# preemption
for i in range(len(paper_authors)):
    other_arguments = [arguments[paper_authors[j].id] for j in range(len(paper_authors)) if j != i]
    other_evidence = [evidence[paper_authors[j].id] for j in range(len(paper_authors)) if j != i]

    preemption = paper_authors[i].preempt_arguments(other_arguments, other_evidence)
    
    # logging
    if log is not None:
        with open(os.path.join(log, 'self_deliberation.txt'), 'a') as f:
            f.write(f'Preemption:\n\n')
            f.write(f'{paper_author.focus} paper:\n{preemption}\n\n')

    evidence[paper_authors[i].id].append(preemption)         

# for child_topic in arguments[focus_paper.id]:
#     children.append(DebateNode(child_topic, parent=self))

AttributeError: 'list' object has no attribute 'keys'

: 

In [ ]:
conversation_history = []

queue_of_rounds: List[DebateNode] = []
queue_of_rounds.extend(subtrees)


while len(queue_of_rounds) > 0:
    round = queue_of_rounds.pop(0)
    conversation, new_focus_arg, new_cited_arg = round.conduct_debate(focus_paper, cited_paper)
    conversation_history.extend(conversation)
    if moderator.is_expand(round.arguments, [new_focus_arg, new_cited_arg]):
        new_subtrees = round.conduct_self_deliberation(round.round_topic, paper_authors)
        queue_of_rounds.extend(new_subtrees)

with open('conversation_history.txt', 'w+') as f:
    f.write('\n'.join(conversation_history))